<a href="https://colab.research.google.com/github/thongchaiphumphuang/SGA_Adhoc/blob/main/KCC_create_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from __future__ import print_function
sns.set_theme(style="whitegrid")

pd.options.mode.use_inf_as_na = True
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)

In [ ]:
import pandas_gbq

In [ ]:
def read_data(key, sheet_name):
    wb = gc.open_by_key(key)
    sh = wb.worksheet(sheet_name)
    data = sh.get_all_values()
    df = pd.DataFrame.from_records(data[1:], columns=data[0])
    return df

#Create db CP&A

## Inventory df

In [ ]:
key_google_sheet = '1emy-GXlF6shNgXGBqo_hxFsLxkoVVtbxjZX4fqleOww'
sheet_name = 'Inventory'
inv_df = read_data(key_google_sheet, sheet_name)
inv_df['Date'] = pd.to_datetime(inv_df['Date'],  format= "%d-%m-%Y")
inv_df['Date'] = pd.to_datetime(inv_df['Date'])
inv_df.head(3)

,Date,SKU,Barcode,DESCRIPTION PRODUCT,Packing,ODX_case,ODX_pcs,LPB_case,LPB_pcs,XKH01_case,XKH01_pcs,VTE_case,VTE_pcs,SVK_case,SVK_pcs,CPS_case,CPS_pcs
0,2022-11-01,1524932,8850006272015,DCS 140g,72,0.00,0.00,0.00,0.00,0,0,0.00,0.00,0.00,0.00,0.00,0.00
1,2022-11-01,TH03575A,8850006272015,DCS 140g,48,915.00,35.00,1042.00,25.88,0,0,1800.00,46.00,66.00,45.00,200.00,8.00
2,2022-11-01,61016329,8850006272016,DCS 140g 5+1,48,977.00,6.00,548.00,9.00,0,0,3703.00,11.00,382.00,28.00,430.00,42.00


In [ ]:
inv_df[['Packing','ODX_case',
       'ODX_pcs', 'LPB_case', 'LPB_pcs', 'VTE_case', 'VTE_pcs', 'SVK_case',
       'SVK_pcs', 'CPS_case', 'CPS_pcs', 'XKH01_case', 'XKH01_pcs']] = inv_df[['Packing','ODX_case',
       'ODX_pcs', 'LPB_case', 'LPB_pcs', 'VTE_case', 'VTE_pcs', 'SVK_case',
       'SVK_pcs', 'CPS_case', 'CPS_pcs','XKH01_case', 'XKH01_pcs']].apply(lambda x: x.astype('float64'))

In [ ]:
inv_df['ODX_pcs'] = inv_df['ODX_pcs'] / inv_df['Packing']
inv_df['LPB_pcs'] = inv_df['LPB_pcs'] / inv_df['Packing']
inv_df['VTE_pcs'] = inv_df['VTE_pcs'] / inv_df['Packing']
inv_df['SVK_pcs'] = inv_df['SVK_pcs'] / inv_df['Packing']
inv_df['CPS_pcs'] = inv_df['CPS_pcs'] / inv_df['Packing']
inv_df['XKH01_pcs'] = inv_df['XKH01_pcs'] / inv_df['Packing']

In [ ]:

inv_df['ODX_case'] = inv_df['ODX_case']  +  inv_df['ODX_pcs']
inv_df['LPB_case'] = inv_df['LPB_case']+ inv_df['LPB_pcs']
inv_df['VTE_case'] = inv_df['VTE_case'] + inv_df['VTE_pcs']
inv_df['SVK_case'] = inv_df['SVK_case'] + inv_df['SVK_pcs']
inv_df['CPS_case'] = inv_df['CPS_case'] + inv_df['CPS_pcs']
inv_df['XKH01_case'] = inv_df['XKH01_case'] + inv_df['XKH01_pcs']

In [ ]:
inv_df = inv_df[['Date', 'SKU', 'ODX_case', 'LPB_case', 'VTE_case', 'SVK_case', 'CPS_case','XKH01_case']]

In [ ]:
inv_df.columns = ['Date', 'SKU', 'ODX', 'LPB', 'VTE', 'SVK', 'CPS','XKH01']

In [ ]:
inv_df = inv_df.melt(id_vars=['Date', 'SKU'], value_vars=['ODX', 'LPB', 'VTE', 'SVK', 'CPS','XKH01'], var_name='Warehouse', value_name='INV')

In [ ]:
inv_df.head()

,Date,SKU,Warehouse,INV
0,2022-11-01,1524932,ODX,0.00
1,2022-11-01,TH03575A,ODX,915.73
2,2022-11-01,61016329,ODX,977.12
3,2022-11-01,1520163,ODX,8.83
4,2022-11-01,61003012,ODX,0.00


## SI df

In [ ]:
key_google_sheet = '1emy-GXlF6shNgXGBqo_hxFsLxkoVVtbxjZX4fqleOww'
sheet_name = 'Sell-In'
si_df = read_data(key_google_sheet, sheet_name)
# si_df = si_df[["Date", "SKU No.", "SKU Description", "Unit (CTN)"]]
si_df['Date'] = pd.to_datetime(si_df['Date'], format= "%d-%m-%Y")
# si_df['Unit (CTN)'] = si_df['Unit (CTN)'].replace('', 0).astype('float')
# si_df = si_df.groupby(['Date','SKU No.'], as_index=False)[['Unit (CTN)']].sum()
# si_df.columns = ['Date','SKU','SI']
# si_df['Warehouse'] = "-"
si_df.tail()

,Date,SKU,ODX,LPB,XKH01,VTE,SVK,CPS
533,2023-04-01,TH03148A,0,5,0,0,30,25
534,2023-04-01,61011583,0,10,0,0,10,15
535,2023-04-01,61019982,0,0,0,0,0,0
536,2023-04-01,61030793,0,0,0,80,30,20
537,2023-04-01,1506791,0,0,0,0,0,0


In [ ]:
si_df  = pd.melt(si_df, id_vars=['Date', 'SKU'], value_vars=['ODX', 'LPB', 'XKH01', 'VTE', 'SVK', 'CPS'], var_name = "Warehouse",value_name="SI")

In [ ]:
si_df['SI'] = si_df['SI'].astype('float64')

In [ ]:
si_df

,Date,SKU,Warehouse,SI
0,2023-01-01,1524932,ODX,0.00
1,2023-01-01,TH03575A,ODX,0.00
2,2023-01-01,61016329,ODX,0.00
3,2023-01-01,1520163,ODX,0.00
4,2023-01-01,61003012,ODX,0.00
...,...,...,...,...
3223,2023-04-01,TH03148A,CPS,25.00
3224,2023-04-01,61011583,CPS,15.00
3225,2023-04-01,61019982,CPS,0.00
3226,2023-04-01,61030793,CPS,20.00


## SO df

In [ ]:
key_google_sheet = '1emy-GXlF6shNgXGBqo_hxFsLxkoVVtbxjZX4fqleOww'
sheet_name = 'Sell-Out'
so_df = read_data(key_google_sheet, sheet_name)
so_df['Date'] = pd.to_datetime(so_df['Date'],  format= "%d-%m-%Y")
so_df[['Sell out', 'FOC']] = so_df[['Sell out', 'FOC']].astype('float64')

warehouse = {"A" : "ODX", "B" : "LPB","B" : "XKH01","C" : "VTE","D" : "SVK","E" : "CPS"}
so_df['Sell out']  = so_df['Sell out'] + so_df['FOC']  
so_df['Warehouse'] = so_df['Zone'].map(warehouse)
so_df.head()

,Date,Zone,Barcode,Sell out,FOC,Warehouse
0,2022-11-01,A,8850006272015.00,0.00,0.00,ODX
1,2022-11-01,A,8850006272015.48,1.00,0.00,ODX
2,2022-11-01,A,8850006272015.60,214.00,0.00,ODX
3,2022-11-01,A,8850006320235.00,0.00,0.00,ODX
4,2022-11-01,A,8850006274019.00,68.00,13.00,ODX


# Barcode -> SKU

In [ ]:
key_google_sheet = '1emy-GXlF6shNgXGBqo_hxFsLxkoVVtbxjZX4fqleOww'
sheet_name = 'Index-SKU'
sku_df = read_data(key_google_sheet, sheet_name)
sku_df.head()

,Barcode,SKU
0,8850006272015.12,1524932
1,8850006272015.48,TH03575A
2,8850006272015.60,61016329
3,8850006320235.00,1520163
4,8850006272015.00,61003012


In [ ]:
so_df.head()

,Date,Zone,Barcode,Sell out,FOC,Warehouse
0,2022-11-01,A,8850006272015.00,0.00,0.00,ODX
1,2022-11-01,A,8850006272015.48,1.00,0.00,ODX
2,2022-11-01,A,8850006272015.60,214.00,0.00,ODX
3,2022-11-01,A,8850006320235.00,0.00,0.00,ODX
4,2022-11-01,A,8850006274019.00,68.00,13.00,ODX


In [ ]:
sku_df

,Barcode,SKU
0,8850006272015.12,1524932
1,8850006272015.48,TH03575A
2,8850006272015.60,61016329
3,8850006320235.00,1520163
4,8850006272015.00,61003012
...,...,...
131,8850006675052.00,TH03148A
132,8850006938720.00,61011583
133,8850006943731.00,61019982
134,8850006947326.00,61030793


In [ ]:
merged_df = so_df.merge(sku_df, on = ['Barcode'], how = 'left')
# merged_df
so_vol_df = merged_df.rename(columns={"Date": "Date", "Warehouse": "Warehouse", "SKU" : "SKU", "Sell out" : "SO"})
so_vol_df = so_vol_df[['Date', "Warehouse", "SKU", "SO"]]
so_vol_df.tail()

,Date,Warehouse,SKU,SO
3895,2023-04-01,CPS,TH03148A,24.09
3896,2023-04-01,CPS,61011583,8.59
3897,2023-04-01,CPS,61030793,12.67
3898,2023-04-01,CPS,61019982,3.00
3899,2023-04-01,CPS,1506791,0.00


In [ ]:
so_df = so_vol_df[['Date', "Warehouse", "SKU", "SO"]]
so_df['SO'] = so_df['SO'].astype('float64')
so_df.tail()

,Date,Warehouse,SKU,SO
3895,2023-04-01,CPS,TH03148A,24.09
3896,2023-04-01,CPS,61011583,8.59
3897,2023-04-01,CPS,61030793,12.67
3898,2023-04-01,CPS,61019982,3.00
3899,2023-04-01,CPS,1506791,0.00


In [ ]:
so_df.groupby(['Date'])[['SO']].sum()

,SO
Date,
2022-11-01,"33,520.00"
2022-12-01,"40,593.90"
2023-01-01,"52,289.50"
2023-02-01,"36,052.50"
2023-03-01,"34,744.00"
2023-04-01,"41,357.73"


## Price df

In [ ]:
key_google_sheet = "1emy-GXlF6shNgXGBqo_hxFsLxkoVVtbxjZX4fqleOww"
sheet_name = 'Index-Price'
price_df = read_data(key_google_sheet, sheet_name)
price_df = price_df[['SKU', 'Price/Ctn']]
price_df['Price/Ctn']= price_df['Price/Ctn'].astype('float64')
price_df

,SKU,Price/Ctn
0,1524932,"3,072.00"
1,TH03575A,"2,051.00"
2,61016329,"1,708.00"
3,1520163,"2,928.00"
4,61003012,"3,072.00"
...,...,...
131,TH03148A,"1,112.00"
132,61011583,"1,112.00"
133,61019982,"1,112.00"
134,61030793,"1,112.00"


In [ ]:
merged_df = inv_df \
            .merge(so_df, on = ["Date", "Warehouse", "SKU"], how='left') \
            .merge(si_df, on = ["Date", "Warehouse", "SKU"], how='left')

In [ ]:
wh_df = merged_df.merge(price_df, left_on  = ['SKU'], right_on =  ['SKU'], how='left')

In [ ]:
wh_df

,Date,SKU,Warehouse,INV,SO,SI,Price/Ctn
0,2022-11-01,1524932,ODX,0.00,NaN,NaN,"3,072.00"
1,2022-11-01,TH03575A,ODX,915.73,1.00,NaN,"2,051.00"
2,2022-11-01,61016329,ODX,977.12,214.00,NaN,"1,708.00"
3,2022-11-01,1520163,ODX,8.83,0.00,NaN,"2,928.00"
4,2022-11-01,61003012,ODX,0.00,0.00,NaN,"3,072.00"
...,...,...,...,...,...,...,...
5617,2023-05-01,TH03148A,XKH01,4.08,NaN,NaN,"1,112.00"
5618,2023-05-01,61011583,XKH01,2.92,NaN,NaN,"1,112.00"
5619,2023-05-01,61019982,XKH01,0.00,NaN,NaN,"1,112.00"
5620,2023-05-01,61030793,XKH01,3.75,NaN,NaN,"1,112.00"


In [ ]:
wh_df['SI (Val)'] = wh_df['SI'] * wh_df['Price/Ctn']
wh_df['SO (Val)'] = wh_df['SO'] * wh_df['Price/Ctn']
wh_df['INV (Val)'] = wh_df['INV'] * wh_df['Price/Ctn']
wh_df = wh_df[["Date", "Warehouse", "SKU", "INV", "SI", "SO", "SI (Val)", "SO (Val)",'INV (Val)']]

In [ ]:
wh_df.groupby(['Date'])[['SO', "SO (Val)"]].sum()

,SO,SO (Val)
Date,,
2022-11-01,"33,346.00","67,106,320.00"
2022-12-01,"40,354.90","63,479,815.60"
2023-01-01,"52,289.50","66,603,101.30"
2023-02-01,"36,052.50","61,560,637.20"
2023-03-01,"34,744.00","65,595,543.30"
2023-04-01,"41,357.73","41,595,749.04"
2023-05-01,0.00,0.00


In [ ]:
project_id = "cp-cdo-prod-ap-self-service"

sql = """
SELECT bic_zcategry, bic_zsubcats, bic_zsubbr , SKU_Number as SKU, ppgtext, product_description FROM `cp-cdo-prod-ap-self-service.AP_DIV_Dataset.TBL_PNL_CALC_AP`
where Sales_Org_Value = "TH01"
group by 1,2,3,4,5,6
"""
df_cate = pandas_gbq.read_gbq(sql, project_id=project_id)
df_cate

Downloading: 100%|██████████|


,bic_zcategry,bic_zsubcats,bic_zsubbr,SKU,ppgtext,product_description
0,Personal Care,Liquid Hand Wash,Protex LHW,1507607,PTX LHS 250ml,PTX LHS Family 250ml(Opaque)
1,Personal Care,Conditioner,Palmolive Micellar Conditioner,61007343,,PO CON MANUKA HONEY 370ML x24(SRP)
2,Personal Care,Body Wash,Pack Matl Body Wash SB BW,P1560083,,Cap PO Heart 250ml Green-Orf 6mm
3,Oral Care,Manual TB,Colgate Bamboo Man TB,61004076,Colgate Bamboo Man TB 1pk,CTBM Bamboo Charcoal Pack1 TH*36
4,Personal Care,Liquid Hand Wash,Palmolive Luminous Oils LHW,61000831,,PO LHW LUM OIL FIG OIL 500MLX12_IN
...,...,...,...,...,...,...
5481,Oral Care,Toothpaste,Colgate Maximum Cavity Protection TP,1524922,Big Red 145/180g MP 47B,COL CDC TP DCS 145G MP 47B
5482,Oral Care,Toothpaste,Colgate Maximum Cavity Protection TP,TH03536A,Big Red 145/150/170/180g Twin,COL CDC TP FCM 170g Twin (RL)
5483,Oral Care,Toothpaste,Colgate Maximum Cavity Protection TP,TH03620A,,CDC RED FCM 250g (RL)_MY
5484,Oral Care,Toothpaste,Colgate Maximum Cavity Protection TP,TH02164A,Big Red 80/90g + Big Red 20/25g,COL CDC FCM 90G + GRF 20g


In [ ]:
db_df = wh_df.merge(df_cate, how='left', on='SKU')

In [ ]:
db_df.groupby(['Date'])[['SO', 'SO (Val)']].sum()

,SO,SO (Val)
Date,,
2022-11-01,"33,346.00","67,106,320.00"
2022-12-01,"40,354.90","63,479,815.60"
2023-01-01,"52,289.50","66,603,101.30"
2023-02-01,"36,052.50","61,560,637.20"
2023-03-01,"34,744.00","65,595,543.30"
2023-04-01,"41,357.73","41,595,749.04"
2023-05-01,0.00,0.00


In [ ]:
db_df = db_df[['Date', 'Warehouse', 'SKU', 'bic_zcategry', 'bic_zsubcats', 'bic_zsubbr', 'product_description',"SI","SO","INV", "SI (Val)", "SO (Val)", "INV (Val)"]]

In [ ]:
db_df["Date"] = db_df["Date"].astype("datetime64[M]")

In [ ]:
db_df[['bic_zcategry', 'bic_zsubcats', 'bic_zsubbr', 'product_description']] = db_df[['bic_zcategry', 'bic_zsubcats', 'bic_zsubbr', 'product_description']] .fillna("-")

In [ ]:
db_df_pm = db_df.copy()
db_df_pm['Date']  = db_df_pm['Date'] + pd.DateOffset(months=1)
db_df_pm[['bic_zcategry', 'bic_zsubcats', 'bic_zsubbr', 'product_description']] = db_df[['bic_zcategry', 'bic_zsubcats', 'bic_zsubbr', 'product_description']] .fillna("-")
db_df_pm.head()

,Date,Warehouse,SKU,bic_zcategry,bic_zsubcats,bic_zsubbr,product_description,SI,SO,INV,SI (Val),SO (Val),INV (Val)
0,2022-12-01,ODX,1524932,Oral Care,Toothpaste,Colgate Maximum Cavity Protection TP,COL TP DCS 140g IDC,NaN,NaN,0.00,NaN,NaN,0.00
1,2022-12-01,ODX,TH03575A,Oral Care,Toothpaste,Colgate Maximum Cavity Protection TP,COL CDC TP DCS 140g (RL),NaN,1.00,915.73,NaN,"2,051.00","1,878,160.52"
2,2022-12-01,ODX,61016329,Oral Care,Toothpaste,Colgate Maximum Cavity Protection TP,COL CDC TP DCS 140g 5+1,NaN,214.00,977.12,NaN,"365,512.00","1,668,929.50"
3,2022-12-01,ODX,1520163,Oral Care,Toothpaste,Colgate Maximum Cavity Protection TP,COL TP CDC DCS 140g Twin,NaN,0.00,8.83,NaN,0.00,"25,864.00"
4,2022-12-01,ODX,61003012,Oral Care,Toothpaste,Colgate Maximum Cavity Protection TP,Col DCS 160g Bonus pack (Laos/CBD),NaN,0.00,0.00,NaN,0.00,0.00


In [ ]:
db_df_pm.groupby(['Date'])['SI'].sum()

Date
2022-12-01        0.00
2023-01-01        0.00
2023-02-01   43,742.00
2023-03-01   49,212.00
2023-04-01   37,678.00
2023-05-01   33,085.00
2023-06-01        0.00
Name: SI, dtype: float64

In [ ]:
db_df_2pm = db_df.copy()
db_df_2pm['Date']  = db_df_2pm['Date'] + pd.DateOffset(months=2)
db_df_2pm[['bic_zcategry', 'bic_zsubcats', 'bic_zsubbr', 'product_description']] = db_df[['bic_zcategry', 'bic_zsubcats', 'bic_zsubbr', 'product_description']] .fillna("-")

db_df_2pm.head()

,Date,Warehouse,SKU,bic_zcategry,bic_zsubcats,bic_zsubbr,product_description,SI,SO,INV,SI (Val),SO (Val),INV (Val)
0,2023-01-01,ODX,1524932,Oral Care,Toothpaste,Colgate Maximum Cavity Protection TP,COL TP DCS 140g IDC,NaN,NaN,0.00,NaN,NaN,0.00
1,2023-01-01,ODX,TH03575A,Oral Care,Toothpaste,Colgate Maximum Cavity Protection TP,COL CDC TP DCS 140g (RL),NaN,1.00,915.73,NaN,"2,051.00","1,878,160.52"
2,2023-01-01,ODX,61016329,Oral Care,Toothpaste,Colgate Maximum Cavity Protection TP,COL CDC TP DCS 140g 5+1,NaN,214.00,977.12,NaN,"365,512.00","1,668,929.50"
3,2023-01-01,ODX,1520163,Oral Care,Toothpaste,Colgate Maximum Cavity Protection TP,COL TP CDC DCS 140g Twin,NaN,0.00,8.83,NaN,0.00,"25,864.00"
4,2023-01-01,ODX,61003012,Oral Care,Toothpaste,Colgate Maximum Cavity Protection TP,Col DCS 160g Bonus pack (Laos/CBD),NaN,0.00,0.00,NaN,0.00,0.00


In [ ]:
a = db_df.groupby(["Date", "Warehouse", "SKU","bic_zcategry", 'bic_zsubcats', 'bic_zsubbr', 'product_description'])[["SI","SO","INV", "SI (Val)", "SO (Val)", "INV (Val)"]].sum()
b = db_df_pm.groupby(["Date", "Warehouse", "SKU","bic_zcategry", 'bic_zsubcats', 'bic_zsubbr', 'product_description'])[["SI","SO","INV", "SI (Val)", "SO (Val)", "INV (Val)"]].sum()
c = db_df_2pm.groupby(["Date", "Warehouse", "SKU","bic_zcategry", 'bic_zsubcats', 'bic_zsubbr', 'product_description'])[["SI","SO","INV", "SI (Val)", "SO (Val)", "INV (Val)"]].sum()
df  = a.merge(b, left_index=True, right_index =True, how='outer').merge(c, left_index=True, right_index =True, how='outer').reset_index()

In [ ]:
df  = df[["Date", "Warehouse", "SKU","bic_zcategry", 'bic_zsubcats', 'bic_zsubbr', 'product_description', 'SI_x', 'SI (Val)_x',
          'INV_x','INV_y', 'SO_x',  'SO_y', 'SO',
           'SO (Val)_x','SO (Val)_y', 'SO (Val)' ,'INV (Val)_x', 'INV (Val)_y']]
df.columns = ["Date", "Warehouse", "SKU","bic_zcategry", 'bic_zsubcats', 'bic_zsubbr', 'product_description', 'SI_Vol', "SI_Val",
              'INV_Vol','INV_Vol_pp',  'SO_Vol', 'SO_Vol_pp', 'SO_Vol_ppp'
            , "SO_Val", "SO_Val_pp", 'SO_Val_ppp',"INV_Val", 'INV_Val_pp' ]

In [ ]:
df.head()

,Date,Warehouse,SKU,bic_zcategry,bic_zsubcats,bic_zsubbr,product_description,SI_Vol,SI_Val,INV_Vol,INV_Vol_pp,SO_Vol,SO_Vol_pp,SO_Vol_ppp,SO_Val,SO_Val_pp,SO_Val_ppp,INV_Val,INV_Val_pp
0,2022-11-01,CPS,1050448,Oral Care,Manual TB,Colgate Classic Man TB,CTBM DELUXE PLUS FH M 1PK THx72,0.00,0.00,637.85,NaN,198.00,NaN,NaN,"117,810.00",NaN,NaN,"379,519.10",NaN
1,2022-11-01,CPS,1051118,Oral Care,Manual TB,Colgate Slim Soft Man TB,CTBM SS Gentle Clean CH XS 1PK Asia x72,0.00,0.00,12.67,NaN,2.00,NaN,NaN,"2,138.00",NaN,NaN,"13,540.67",NaN
2,2022-11-01,CPS,1051803,Oral Care,Manual TB,Colgate Classic Man TB,CTBM DELUXE CHILD 5-9 CH XS 1PK THX72,0.00,0.00,19.67,NaN,6.00,NaN,NaN,"2,316.00",NaN,NaN,"7,591.33",NaN
3,2022-11-01,CPS,1051826,Oral Care,Manual TB,Colgate Kids Man TB,CTBM KIDS MID TIER 5+ CH XS 1PK ASIA*72,0.00,0.00,7.17,NaN,0.00,NaN,NaN,0.00,NaN,NaN,"9,001.33",NaN
4,2022-11-01,CPS,1053300,-,-,-,-,0.00,0.00,0.00,NaN,0.00,NaN,NaN,0.00,NaN,NaN,0.00,NaN


In [ ]:
df.groupby(['Date', 'Warehouse']).sum()

<ipython-input-192-7ee36be6386e>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby(['Date', 'Warehouse']).sum()


SI_Vol        SI_Val   INV_Vol  INV_Vol_pp    SO_Vol  \
Date       Warehouse                                                           
2022-11-01 CPS            0.00          0.00  7,455.23        0.00  2,696.00   
           LPB            0.00          0.00  9,796.98        0.00      0.00   
           ODX            0.00          0.00  9,601.10        0.00  2,696.00   
           SVK            0.00          0.00 10,667.43        0.00 13,054.00   
           VTE            0.00          0.00 47,718.04        0.00 12,204.00   
           XKH01          0.00          0.00      0.00        0.00  2,696.00   
2022-12-01 CPS            0.00          0.00  5,973.56    7,455.23  1,942.00   
           LPB            0.00          0.00  6,828.14    9,796.98      0.00   
           ODX            0.00          0.00  8,224.23    9,601.10  1,942.00   
           SVK            0.00          0.00  8,752.13   10,667.43 20,329.00   
           VTE            0.00          0.00 41,979.54   47,718.04 12,941.00   
           XKH01          0.00          0.00      0.00        0.00  3,200.90   
2023-01-01 CPS        2,924.00  4,184,049.00  6,108.87    5,973.56  2,197.00   
           LPB        1,704.00  2,553,798.00  6,324.18    6,828.14      0.00   
           ODX            0.00          0.00  7,610.56    8,224.23  2,197.00   
           SVK            0.00          0.00  6,649.42    8,752.13 18,767.00   
           VTE       39,114.00 61,936,324.00 37,172.91   41,979.54 27,118.00   
           XKH01          0.00          0.00      0.00        0.00  2,010.50   
2023-02-01 CPS            0.00          0.00  3,274.22    6,108.87  3,257.00   
           LPB        2,890.00  5,216,909.00  6,260.21    6,324.18      0.00   
           ODX            0.00          0.00  5,323.91    7,610.56  3,257.00   
           SVK        4,166.00  6,512,140.00  8,208.66    6,649.42 13,039.00   
           VTE       42,156.00 52,236,466.00 54,775.09   37,172.91 12,113.00   
           XKH01          0.00          0.00      0.00        0.00  4,386.50   
2023-03-01 CPS        4,040.00  7,097,893.00  5,235.13    3,274.22  3,278.00   
           LPB        3,140.00  4,261,326.00  4,655.54    6,260.21      0.00   
           ODX        1,405.00  2,376,054.00  5,599.51    5,323.91  3,278.00   
           SVK        1,448.00  2,206,356.00  4,143.06    8,208.66 11,108.00   
           VTE       27,645.00 45,412,010.00 60,940.43   54,775.09 12,193.00   
           XKH01          0.00          0.00  1,093.51        0.00  4,887.00   
2023-04-01 CPS        2,788.00  3,658,235.00  6,796.78    5,235.13  1,845.74   
           LPB        3,166.00  3,927,109.00  5,771.64    4,655.54      0.00   
           ODX        1,348.00  2,332,967.00  4,438.81    5,599.51  3,202.86   
           SVK        3,043.00  3,581,523.00  4,924.02    4,143.06  1,580.44   
           VTE       22,740.00 39,189,833.00 50,183.97   60,940.43 32,008.59   
           XKH01          0.00          0.00  1,559.20    1,093.51  2,720.10   
2023-05-01 CPS            0.00          0.00  4,640.48    6,796.78      0.00   
           LPB            0.00          0.00  6,450.97    5,771.64      0.00   
           ODX            0.00          0.00  5,760.44    4,438.81      0.00   
           SVK            0.00          0.00  6,382.99    4,924.02      0.00   
           VTE            0.00          0.00 49,798.40   50,183.97      0.00   
           XKH01          0.00          0.00  1,463.55    1,559.20      0.00   
2023-06-01 CPS            0.00          0.00      0.00    4,640.48      0.00   
           LPB            0.00          0.00      0.00    6,450.97      0.00   
           ODX            0.00          0.00      0.00    5,760.44      0.00   
           SVK            0.00          0.00      0.00    6,382.99      0.00   
           VTE            0.00          0.00      0.00   49,798.40      0.00   
           XKH01          0.00          0.00      0.00    1,463.55      0.00   
2023-07-01 CPS          

In [ ]:
df.groupby(['Date', 'Warehouse']).sum().loc['2023-05-01']

<ipython-input-193-1378066cb7b4>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby(['Date', 'Warehouse']).sum().loc['2023-05-01']


,SI_Vol,SI_Val,INV_Vol,INV_Vol_pp,SO_Vol,SO_Vol_pp,SO_Vol_ppp,SO_Val,SO_Val_pp,SO_Val_ppp,INV_Val,INV_Val_pp
Warehouse,,,,,,,,,,,,
CPS,0.00,0.00,"4,640.48","6,796.78",0.00,"1,845.74","3,278.00",0.00,"2,965,045.19","5,491,005.00","8,618,405.13","12,139,109.36"
LPB,0.00,0.00,"6,450.97","5,771.64",0.00,0.00,0.00,0.00,0.00,0.00,"9,962,009.83","8,544,815.40"
ODX,0.00,0.00,"5,760.44","4,438.81",0.00,"3,202.86","3,278.00",0.00,"4,382,545.58","5,491,005.00","9,204,861.82","7,553,247.51"
SVK,0.00,0.00,"6,382.99","4,924.02",0.00,"1,580.44","11,108.00",0.00,"2,555,202.13","26,144,069.00","10,238,280.34","9,239,267.73"
VTE,0.00,0.00,"49,798.40","50,183.97",0.00,"32,008.59","12,193.00",0.00,"27,186,006.24","21,665,591.00","78,999,833.21","102,555,278.76"
XKH01,0.00,0.00,"1,463.55","1,559.20",0.00,"2,720.10","4,887.00",0.00,"4,506,949.90","6,803,873.30","2,157,747.58","2,377,103.25"


In [ ]:
#@title Load datafram to BigQuery
import datetime

from google.cloud import bigquery
import pandas
import pytz

# Construct a BigQuery client object.
client = bigquery.Client(project ='cp-cdo-prod-ap-self-service')

# TODO(developer): Set table_id to the ID of the table to create.
table_id = "cp-cdo-prod-ap-self-service.Thailand.TBL_CDT_INV_INDO_KCC"

job_config = bigquery.LoadJobConfig(
    # Specify a (partial) schema. All columns are always written to the
    # table. The schema is used to assist in data type definitions.
    schema=[
        # Specify the type of columns whose type cannot be auto-detected. For
        # example the "title" column uses pandas dtype "object", so its
        # data type is ambiguous.
        bigquery.SchemaField("Warehouse", bigquery.enums.SqlTypeNames.STRING),
        # Indexes are written if included in the schema by name.
        bigquery.SchemaField("SKU", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("bic_zcategry", bigquery.enums.SqlTypeNames.STRING)
    ],
    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    write_disposition="WRITE_TRUNCATE",
)

job = client.load_table_from_dataframe(
    df, table_id, job_config=job_config
)  # Make an API request.
job.result()  # Wait for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

Loaded 7290 rows and 19 columns to cp-cdo-prod-ap-self-service.Thailand.TBL_CDT_INV_INDO_KCC
